In [10]:
using Pkg
Pkg.activate(joinpath(@__DIR__, ".."))

"/home/twan/.julia/dev/PushRecovery/Project.toml"

In [11]:
using LinearAlgebra
using QPControl
using RigidBodyDynamics
using RigidBodyDynamics.PDControl
using RigidBodyDynamics.Contact
using StaticArrays
using AtlasRobot
using Test
using RigidBodySim
using PushRecovery
BLAS.set_num_threads(1)

In [12]:
@time AtlasRobot.mechanism(add_flat_ground=true);

  0.005430 seconds (46.80 k allocations: 1.877 MiB)


In [13]:
@time mechanism = AtlasRobot.mechanism(add_flat_ground=true);

  0.005462 seconds (46.81 k allocations: 1.943 MiB)


In [14]:
# create optimizer
using MathOptInterface
using OSQP
using OSQP.MathOptInterfaceOSQP: OSQPSettings
const MOI = MathOptInterface
optimizer = OSQP.Optimizer()
MOI.set(optimizer, OSQPSettings.Verbose(), false)
MOI.set(optimizer, OSQPSettings.EpsAbs(), 1e-5)
MOI.set(optimizer, OSQPSettings.EpsRel(), 1e-5)
MOI.set(optimizer, OSQPSettings.MaxIter(), 5000)
MOI.set(optimizer, OSQPSettings.AdaptiveRhoInterval(), 25) # required for deterministic behavior

In [15]:
# create low level controller
num_basis_vectors = 4
lowlevel = MomentumBasedController{num_basis_vectors}(mechanism, optimizer,
    floatingjoint = findjoint(mechanism, "pelvis_to_world"));
for body in bodies(mechanism)
    for point in RigidBodyDynamics.contact_points(body)
        position = location(point)
        normal = FreeVector3D(default_frame(body), 0.0, 0.0, 1.0)
        μ = point.model.friction.μ
        contact = addcontact!(lowlevel, body, position, normal, μ)
        contact.maxnormalforce[] = 1e6 # TODO
        contact.weight[] = 1e-3
    end
end

In [16]:
# create high level controller
feet = findbody.(Ref(mechanism), ["l_foot", "r_foot"])
pelvis = findbody(mechanism, "pelvis")
nominalstate = MechanismState(mechanism)
AtlasRobot.setnominal!(nominalstate)
controller = PushRecoveryController(lowlevel, feet, pelvis, nominalstate);

In [49]:
# create push applier
pushapplier = PushApplier(mechanism, Point3D(default_frame(pelvis), 0.0, 0.0, 0.0));

In [66]:
# create visualizer
using MeshCat
using MeshCatMechanisms
if !(@isdefined gui)
    vis = Visualizer()[:atlas]
    visuals = URDFVisuals(AtlasRobot.urdfpath(); package_path = [AtlasRobot.packagepath()])
    mvis = MechanismVisualizer(mechanism, visuals, vis)
    gui = PushRecoveryGUI(mvis, Widget(pushapplier, max_force=100.0, max_Δt=0.3))
    set_configuration!(mvis, configuration(nominalstate))
    open(gui)
end

In [67]:
# simulate
state = MechanismState(mechanism)
AtlasRobot.setnominal!(state)
Δt = 1 / 300
pcontroller = PeriodicController(similar(velocity(state)), Δt, controller)
# TODO: add damping
dynamics = Dynamics(mechanism, SumController(similar(velocity(state)), (pcontroller, pushapplier)))
callback = CallbackSet(RealtimeRateLimiter(poll_interval=pi / 100), CallbackSet(gui))
problem = ODEProblem(dynamics, state, (0., Inf); callback=callback)

ODEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, Inf)
u0: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.85, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [68]:
@time sol = solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6);

 25.600472 seconds (24.50 M allocations: 1.557 GiB, 5.85% gc time)


In [69]:
setanimation!(mvis, sol)

In [63]:
# using BenchmarkTools
# AtlasRobot.setnominal!(state)
# τ = similar(velocity(state));
# benchresult = @benchmark $controller($τ, 0.0, $state)
# @test benchresult.allocs == 0
# benchresult